In [6]:
import subprocess
download_command = "wget --directory-prefix downloads https://cricsheet.org/register/people.csv"
subprocess.run(download_command.split(" "))

--2023-02-18 20:23:13--  https://cricsheet.org/register/people.csv
Resolving cricsheet.org (cricsheet.org)... 64.90.49.16
Connecting to cricsheet.org (cricsheet.org)|64.90.49.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 783500 (765K) [text/csv]
Saving to: ‘downloads/people.csv’

     0K .......... .......... .......... .......... ..........  6% 93.0K 8s
    50K .......... .......... .......... .......... .......... 13%  186K 5s
   100K .......... .......... .......... .......... .......... 19% 20.6M 3s
   150K .......... .......... .......... .......... .......... 26% 1.42M 2s
   200K .......... .......... .......... .......... .......... 32%  216K 2s
   250K .......... .......... .......... .......... .......... 39% 11.6M 2s
   300K .......... .......... .......... .......... .......... 45% 5.04M 1s
   350K .......... .......... .......... .......... .......... 52% 1.81M 1s
   400K .......... .......... .......... .......... .......... 58% 3.07M 1s
   4

CompletedProcess(args=['wget', '--directory-prefix', 'downloads', 'https://cricsheet.org/register/people.csv'], returncode=0)

In [11]:
from etl.spark.spark_session_helper import spark
all_players_df = spark.read.csv("downloads/people.csv", header=True)
all_players_df.registerTempTable("all_players")
all_players_df.toPandas()

,identifier,name,unique_name,key_bcci,key_bcci_2,key_bigbash,key_cricbuzz,key_crichq,key_cricinfo,key_cricinfo_2,key_cricingif,key_cricketarchive,key_cricketarchive_2,key_opta,key_opta_2,key_pulse,key_pulse_2
0,b4a23876,AAA Amsterdam,AAA Amsterdam,None,None,None,None,None,772407,None,None,None,None,None,None,None,None
1,4b0e3049,AA Alleyne,AA Alleyne,None,None,None,None,None,661441,None,None,1512811,None,None,None,18542,None
2,fd3c5c00,AAA Patel,AAA Patel,None,None,None,None,None,1163136,None,None,None,None,None,None,None,None
3,bc005f0d,AAA White,AAA White,None,None,None,None,None,474307,None,None,None,None,None,None,None,None
4,5c67c7a6,AA Baig,AA Baig,None,None,None,None,None,524897,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13729,3a0f6df2,Zulfiqar Babar,Zulfiqar Babar,None,None,None,None,None,43871,None,None,45107,None,None,None,984,None
13730,9bf23c13,Zulqarnain,Zulqarnain,None,None,None,None,None,43707,None,None,1803,None,None,None,None,None
13731,ee9bdbc8,Zulqarnain Haider,Zulqarnain Haider,None,None,None,None,None,43860,None,None,45108,None,None,None,None,None
13732,2d46e8ed,Zulqarnain Haider,Zulqarnain Haider (2),None,None,None,None,None,1046641,None,None,None,None,None,None,None,None


In [10]:
player_last_names = [
    "healy",
    "mooney",
    "jafta",
    "tucker",
    "lanning",
    "wolvaardt",
    "luus",
    "mcgrath",
    "bosch",
    "harris",
    "goodall",
    "brits",
    "dercksen",
    "macheke",
    "perry",
    "kapp",
    "gardner",
    "klerk",
    "tryon",
    "sutherland",
    "garth",
    "andrews",
    "schutt",
    "jonassen",
    "ismail",
    "klaas",
    "king",
    "brown",
    "khaka",
    "wareham",
    "mlaba",
    "graham",
    "sekhumkhune"
]

In [24]:
from string import Template
name_predicates = []
name_clause_template = Template("lower(reverse(split(unique_name,' '))[0]) = '$player_name'")
for name in player_last_names:
    name_predicates.append(name_clause_template.substitute(player_name=name))
name_clause = " OR ".join(name_predicates)

In [27]:
player_ids= [ row[0] for row in spark.sql("select identifier, unique_name from all_players where %s"%name_clause).collect()]
player_ids

['1d45c01a',
 'dcf0e8f4',
 '4157fefb',
 'ef14c0ec',
 '278b5e2b',
 'd2b1f515',
 'bc969efb',
 '9b04a912',
 'bbafa5d6',
 '1db5e9db',
 '321644de',
 '0d518f89',
 '83558266',
 'ba0a8d88',
 'e8b5ea8d',
 '65d9b6b6',
 '90d95b82',
 '7fca84b7',
 'bb32a752',
 '8af86756',
 '52d1dbc8',
 'f12d65ee',
 '172dff15',
 'd8cd0b9b',
 'd504bc4c',
 '1b4a7112',
 '4d4605e6',
 '448f4f1c',
 '152d3c9d',
 '914e40a6',
 'dd639a8c',
 '582e9baa',
 '2e81be88',
 '0b4b539d',
 '39f82db3',
 '907e30d9',
 '0e8ffd13',
 '7c70083d',
 'b7879e16',
 '2b425403',
 'be150fc8',
 '28d6c44d',
 '0de3060f',
 '3d942cdc',
 '02e96b8c',
 '2708b41c',
 'afb4c2ac',
 '4303d7ce',
 'f5593357',
 '8ea4f42e',
 'd0f5e72d',
 '43223c50',
 '2ad74020',
 'ee7d0c82',
 '0388d3bd',
 '72166006',
 '3cebb01c',
 'f0b4e47d',
 '4090226d',
 'd2eb0b8c',
 '71e2b411',
 '99812d10',
 '4d4ffcf5',
 '541db522',
 'a216a585',
 '0ef22928',
 'f1ba9ed7',
 '2cfbeaf1',
 'f086e9dc',
 'c122ab7f',
 '647e7d44',
 'a941d2d5',
 'f83b6e91',
 'aae9ce90',
 '184512d1',
 '97607576',
 'bed7ecf5',

In [28]:
len(player_ids)

154